# Quantum Circuit Compiler

This is a quantum circuit compiler and optimizer which converts any given Cirq circuit made up of the basic gates: I, S, H, X, Y, Z, RX, RY, RZ, CNOT, CZ into a combination of gates: RX, RZ, and CZ. It also optimizes if those combinations result in a net 0 change in the state of qubit or if the application of multiple gates can be merged to a single gate, for example: `--RX(pi/2)-RX(pi/2)--` is effectively `--RX(pi)--`.

The compiler also supports inclusion of more gates available in Cirq as long as they can be converted to a combination of our fundamental gates. They can simply be added by implementing a `compile_GATE()` function and adding an `elif` condition during compilation for that gate.

Information about how circuits and gates work in Cirq can be found [here](https://cirq.readthedocs.io/en/stable/).

Note: I decided to perform this on a Jupyter Notebook as it will be easier to explain the steps than writing everything as a comment in files. Also, this notebook can easily be converted to a python file but not the other way round. So, here we go.

## Importing required libraries

In [1]:
import cirq
from math import pi

## Creating a sample circuit

`N_QUBITS` is the total number of qubits that will be required by the sample circuit.
We will be creating a list of `cirq.LineQubit()` qubits to apply the circuit upon.

In [2]:
# Creating qubits for the sample circuit
N_QUBITS = 5
qubits = [cirq.LineQubit(i) for i in range(N_QUBITS)]

Creating a very basic circuit with Hadamard, RX, and CNOT gates.

In [3]:
# Creating a sample circuit
input_circuit = cirq.Circuit()

# Applying Hadamard Gate on first 3 qubits
input_circuit.append([cirq.H(qubits[0])])
input_circuit.append([cirq.H(qubits[1])])
input_circuit.append([cirq.H(qubits[2])])

# Applying CNOT Gate on consecutive qubits
for i in range(N_QUBITS - 1):
    input_circuit.append([cirq.CNOT(qubits[i], qubits[i + 1])])

# Applying RX(pi/2) Gate on all qubits
rx = cirq.rx(pi / 2)
for q in qubits:
    # InserStrategy.INLINE will insert the passed operations in a single Moment
    input_circuit.append([rx(q)], cirq.InsertStrategy.INLINE)

# Printing the circuit
input_circuit

0: ───H───@───────────Rx(0.5π)──────────────
          │
1: ───H───X───@───────Rx(0.5π)──────────────
              │
2: ───H───────X───@───Rx(0.5π)──────────────
                  │
3: ───────────────X───@──────────Rx(0.5π)───
                      │
4: ───────────────────X──────────Rx(0.5π)───

Now, in Cirq circuits are executed on a "moment" by "moment" basis where a `moment` is a time slice. All operations sharing a `moment` will be executed at once. So the first `moment`, in our sample circuit above, consists of a Hadamard gate operation on qubits `qubit[0], qubit[1], qubit[2]`. After executing that, the next `moment` made of a CNOT gate operation on `qubit[0]` as control qubit and `qubit[1]` as target qubit will be executed and so on.

## Creating the Compiler
Codeblock below contains the compiler functions for each supported gate. One can simply add a new gate by implementing a new compiler function `compile_GATE()` which takes in the qubits on which to apply the gate and theta if the gate changes a phase; and returns an equivalent combination of RX, RZ and CZ gates.

For example, a CNOT Gate can be obtained by applying a CZ gate and surrounding the target qubit with Hadamard Gates. The function for that will be `compile_CNOT()` written below.

In [4]:
# Available gates for the compiled circuit
def compile_rx(qubit, theta):
    """
    --RX(theta)--
    """
    rx = cirq.rx(theta)
    return rx(qubit)


def compile_rz(qubit, theta):
    """
    --RZ(theta)--
    """
    rz = cirq.rz(theta)
    return rz(qubit)


def compile_CZ(control_q, target_q):
    """
    control_q: --@--
                 |
    target_q:  --@--
    """
    return cirq.CZ(control_q, target_q)


# Gates that were used by the input circuit apart from the ones above
def compile_ry(qubit, theta):
    """
    --RY(theta)-- is quivalent to
    --RX(pi/2)-RZ(theta)-RX(-pi/2)--
    """
    return compile_rx(qubit, pi / 2), compile_rz(qubit, theta), compile_rx(qubit, -pi / 2)


def compile_I(qubit):
    """
    --I-- is equivalent to
    --RX(2*pi)-- or
    --RZ(2*pi)--
    """
    return compile_rx(qubit, 2 * pi)


def compile_H(qubit):
    """
    --H-- is equivalent to
    --RZ(pi/2)-RX(pi/2)-RZ(pi/2)--
    """
    return compile_rz(qubit, pi / 2), compile_rx(qubit, pi / 2), compile_rz(qubit, pi / 2)


def compile_X(qubit):
    """
    --X-- is equivalent to
    --RX(pi)--
    """
    return compile_rx(qubit, pi)


def compile_Y(qubit):
    """
    --Y-- is equivalent to
    --RY(pi)--
    """
    return compile_ry(qubit, pi)


def compile_Z(qubit):
    """
    --Z-- is equivalent to
    --RZ(pi)--
    """
    return compile_rz(qubit, pi)


def compile_S(qubit):
    """
    --S-- is equivalent to
    --RZ(pi/2)--
    """
    return compile_rz(qubit, pi / 2)


def compile_CNOT(control_q, target_q):
    """
    control_q: --@--
                 |
    target_q:  --X--
    
    is equivalent to

    control_q: ----@----
                   |
    target_q:  --H-@-H--
    """
    return compile_H(target_q), compile_CZ(control_q, target_q), compile_H(target_q)

This method will detect the gate operation being performed in a moment and call the respective `compile_GATE()` function. It then return the compiled gate operations.

More gates can be added by adding an `elif` statement here and passing the required parameters to the `compile_GATE()` function accordingly.

`op.gate` returns the gate that was applied to the qubit(s) by the operation `op` and `op.qubits` returns the list of qubits upon which the operation `op` was performed. For more information on operations objects click [here](https://cirq.readthedocs.io/en/stable/generated/cirq.GateOperation.html).

In [5]:
def get_compiled_ops(moment):
    """
    input: cirq.Moment object
    output: list of cirq.GateOperation objects

    Replaces the operations in given moment to
    a combination of fundamental gates:
    RX, RZ, and CZ 
    """
    compiled_operations = []
    
    # For each operation in a Moment
    for op in moment.operations:

        # Retrieving which gate operation is being performed
        gate = op.gate

        # Compiling gates accordingly:
        # Single qubit gates
        if gate == cirq.I:
            new_op = compile_I(op.qubits[0])

        elif gate == cirq.H:
            new_op = compile_H(op.qubits[0])

        elif gate == cirq.X:
            new_op = compile_X(op.qubits[0])

        elif gate == cirq.Y:
            new_op = compile_Y(op.qubits[0])

        elif gate == cirq.Z:
            new_op = compile_Z(op.qubits[0])
        
        elif gate == cirq.S:
            new_op = compile_S(op.qubits[0])
        
        # Two qubit gates
        elif gate == cirq.CNOT:
            new_op = compile_CNOT(op.qubits[0], op.qubits[1])

        elif gate == cirq.CZ:
            new_op = compile_CZ(op.qubits[0], op.qubits[1])
        
        # Single qubit rotation gates around x, y, z axis on Bloch sphere
        # with rotation by theta radians
        else:

            # Retrieveing the value of theta
            theta = gate.exponent * pi

            if gate == cirq.rx(theta):
                new_op = compile_rx(op.qubits[0], theta)

            elif gate == cirq.ry(theta):
                new_op = compile_ry(op.qubits[0], theta)

            elif gate == cirq.rz(theta):
                new_op = compile_rz(op.qubits[0], theta)
            
            else:
                # If it is an unrecognized gate then leave as is
                new_op = op


        # Add the new operation to the list of compiled operations
        compiled_operations.append(new_op)
        
    return compiled_operations

Compiling the circuit on a moment by moment basis.

In [8]:
compiled_circuit = cirq.Circuit()

# For each moment in input_circuit get compiled operations
for moment in input_circuit:
    compiled_circuit.append(get_compiled_ops(moment))

# Printing the compiled circuit
compiled_circuit

0: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)────────────────────────────────────@───Rx(0.5π)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                        │
1: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)──────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                             │
2: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)─────────────────────────────────────────────────────────────────────────
                                                                                                                                                  │
3: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)──────────────────────────────────────────────────────────────────────────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)────────────────────────────────────
                                                                                                                                                                                       │
4: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───Rx(0.5π)───

In addition to the decreased visual appeal, our compiled circuit has more moments than our input circuit. This increase in the number of moments can be considered as the overhead generated by our compiler. So, let's see how we can decrease this number to restrict its impact on performance when executing the circuit.

In [10]:
print("Number of Moments in the input circuit: ", len(input_circuit))
print("Number of Moments in the compiled output circuit: ", len(compiled_circuit))

Number of Moments in the input circuit:  6
Number of Moments in the compiled output circuit:  23


In [9]:
# Gate combinations that can be merged to I, X, Z. In reversed order for comparison to a stack
# More quantum circuit identities can be added here

can_be_merged_to_i = [
    [cirq.rx(2 * pi)],                                                                 # equivalent to --X-X--
    [cirq.rx(-pi / 2), cirq.rz(pi), cirq.rx(0), cirq.rz(pi), cirq.rx(pi / 2)],         # equivalent to --Y-Y--
    [cirq.rz(2 * pi)],                                                                 # equivalent to --Z-Z--
    [cirq.rz(pi), cirq.rx(-pi / 2), cirq.rz(pi), cirq.rx(3 * pi / 2)],                 # equivalent to reversed --X-Y-Z--
    [cirq.rz(pi / 2), cirq.rx(pi / 2), cirq.rz(pi), cirq.rx(pi / 2), cirq.rz(pi / 2)], # equivalent to --H-H--
]

can_be_merged_to_x = [
    [cirq.rz(pi / 2), cirq.rx(pi), cirq.rz(pi / 2)]                                    # equivalent to --H-Z-H--
]

can_be_merged_to_z = [
    [cirq.rz(pi / 2), cirq.rx(pi / 2), cirq.rz(pi / 2), cirq.rx(pi), cirq.rz(pi / 2), cirq.rx(pi / 2), cirq.rz(pi / 2)]      # equivalent to --H-X-H--
]

Optimizer functions written here.

In [10]:

def merge_similar(ops):
    """
    """
    opt_ops = []                                                # Optimized operations' list
    q = ops[0].qubits[0]                                        # Since these operations are being performed on qubit "q"
    i = 0

    while i < (len(ops) - 1):
        if type(ops[i].gate) == type(ops[i + 1].gate):

            theta_1 = ops[i].gate.exponent * pi                 # theta from first rotation gate
            theta_2 = ops[i + 1].gate.exponent * pi             # theta from second rotation gate
            
            if ops[i].gate == cirq.rx(theta_1):
                opt_gate = cirq.rx(theta_1 + theta_2)
                opt_ops.append(opt_gate(q))
                i += 2
            
            elif ops[i].gate == cirq.rz(theta_1):
                opt_gate = cirq.rz(theta_1 + theta_2)
                opt_ops.append(opt_gate(q))
                i += 2

            else:
                opt_ops.append(ops[i])                       # If two CZ gates were found touching this qubit it will simply add it and move on
                i += 1
        else:
            opt_ops.append(ops[i])
            i += 1
        
    opt_ops.append(ops[-1])                                  # Appending the last operation after checking for merge possibility above

    return opt_ops

def merge_to_one_op(ops):
    """
    
    """
    opt_ops = []
    stack = []
    stack_ops = []

    for operation in ops:
        opt_ops.append(operation)

        while(len(opt_ops) > 0):
            stack_ops.append(opt_ops.pop())
            stack.append(stack_ops[-1].gate)

            if stack in can_be_merged_to_i:
                stack.clear()
                stack_ops.clear()

            elif stack in can_be_merged_to_x:
                stack.clear()
                stack_ops.clear()
                X = cirq.rx(pi)
                stack_ops.append(X(q))
                stack.append(X)

            elif stack in can_be_merged_to_z:
                stack.clear()
                stack_ops.clear()
                Z = cirq.rz(pi)
                stack_ops.append(Z(q))
                stack.append(Z)
        
        stack.clear()
        while(len(stack_ops) > 0):
            opt_ops.append(stack_ops.pop())


    return opt_ops

In [11]:
def optimize(circuit):
    """

    """
    opt_circuit = cirq.Circuit(circuit)
    
    cirq.MergeSingleQubitGates(rewriter=merge_similar).optimize_circuit(opt_circuit)
    cirq.MergeSingleQubitGates(rewriter=merge_to_one_op).optimize_circuit(opt_circuit)
    cirq.DropEmptyMoments().optimize_circuit(opt_circuit)
    
    return opt_circuit

optimized_circuit = optimize(compiled_circuit)

optimized_circuit

0: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                       │
1: ────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)──────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                            │
2: ─────────────────────────────────────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)─────────────────────────────────────────────────────────────────────────
                                                                                                                 │
3: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)─────────────────────────────────────────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)────────────────────────────────────
                                                                                                                                                      │
4: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)──────────────────────────────────────────────────────────────────────────────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───Rx(0.5π)───

Now, after optimization we can see that there has been an improvement, as follows:

In [12]:
print("Number of Moments in the input circuit: ", len(circuit))
print("Number of Moments in the compiled output circuit: ", len(compiled_circuit))
print("Number of Moments in the optimized circuit: ", len(optimized_circuit))

Number of Moments in the input circuit:  6
Number of Moments in the compiled output circuit:  23
Number of Moments in the optimized circuit:  20


As can be seen, after basic optimizations the number of moments were decreased and the optimized circuit is less involved than before. More optimizations can be introduced by simply creating a function which accepts a list of operations on a qubit and return a list of operations on that qubit.

Keep in mind that not all circuits can be improved, for example if a circuit is already simplified by the optimizer's standards then it will not result in a better circuit.

In [13]:
print("Number of Moments in the optimized circuit: ", len(optimized_circuit))
print("Number of Moments in the double optimized circuit: ", len(optimize(optimized_circuit)))

Number of Moments in the optimized circuit:  20
Number of Moments in the double optimized circuit:  20
